#Uber Ride Cancellation Analyst



###Read and Clean Data

In [66]:
import pandas as pd

df = pd.read_csv("uber_rides.csv")

df.head()

,Date,Time,Booking ID,Booking Status,Customer ID,Vehicle Type,Pickup Location,Drop Location,Avg VTAT,Avg CTAT,...,Reason for cancelling by Customer,Cancelled Rides by Driver,Driver Cancellation Reason,Incomplete Rides,Incomplete Rides Reason,Booking Value,Ride Distance,Driver Ratings,Customer Rating,Payment Method
0,2024-03-23,12:29:38,"""CNR5884300""",No Driver Found,"""CID1982111""",eBike,Palam Vihar,Jhilmil,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2024-11-29,18:01:39,"""CNR1326809""",Incomplete,"""CID4604802""",Go Sedan,Shastri Nagar,Gurgaon Sector 56,4.9,14.0,...,NaN,NaN,NaN,1.0,Vehicle Breakdown,237.0,5.73,NaN,NaN,UPI
2,2024-08-23,08:56:10,"""CNR8494506""",Completed,"""CID9202816""",Auto,Khandsa,Malviya Nagar,13.4,25.8,...,NaN,NaN,NaN,NaN,NaN,627.0,13.58,4.9,4.9,Debit Card
3,2024-10-21,17:17:25,"""CNR8906825""",Completed,"""CID2610914""",Premier Sedan,Central Secretariat,Inderlok,13.1,28.5,...,NaN,NaN,NaN,NaN,NaN,416.0,34.02,4.6,5.0,UPI
4,2024-09-16,22:08:00,"""CNR1950162""",Completed,"""CID9933542""",Bike,Ghitorni Village,Khan Market,5.3,19.6,...,NaN,NaN,NaN,NaN,NaN,737.0,48.21,4.1,4.3,UPI


In [67]:
print("Total number of unique rides are {} out of {} rides.".format(len(df['Booking ID'].unique()), len(df)))

## Remove duplicated & invalid rides
df.dropna(subset=['Booking ID'], inplace=True)
df.drop_duplicates(subset=['Booking ID'], inplace=True)

## Combine Date & Time and convert to appropriate type
df['DateTime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'])
df.info()

Total number of unique rides are 148767 out of 150000 rides.
<class 'pandas.core.frame.DataFrame'>
Index: 148767 entries, 0 to 149999
Data columns (total 22 columns):
 #   Column                             Non-Null Count   Dtype         
---  ------                             --------------   -----         
 0   Date                               148767 non-null  object        
 1   Time                               148767 non-null  object        
 2   Booking ID                         148767 non-null  object        
 3   Booking Status                     148767 non-null  object        
 4   Customer ID                        148767 non-null  object        
 5   Vehicle Type                       148767 non-null  object        
 6   Pickup Location                    148767 non-null  object        
 7   Drop Location                      148767 non-null  object        
 8   Avg VTAT                           138366 non-null  float64       
 9   Avg CTAT                           1

###Data Investigation

In [76]:
df['Booking Status'].value_counts(dropna=False)

Booking Status
Completed                92248
Cancelled by Driver      26789
Cancelled by Customer    10402
No Driver Found          10401
Incomplete                8927
Name: count, dtype: int64

In [77]:
df['Booking Status'].value_counts(normalize=True, dropna=False) * 100

Booking Status
Completed                62.008376
Cancelled by Driver      18.007354
Cancelled by Customer     6.992142
No Driver Found           6.991470
Incomplete                6.000659
Name: proportion, dtype: float64

In [70]:
df['Cancelled Rides by Customer'].value_counts()

Cancelled Rides by Customer
1.0    10402
Name: count, dtype: int64

In [71]:
df['Cancelled Rides by Driver'].value_counts()

Cancelled Rides by Driver
1.0    26789
Name: count, dtype: int64

In [72]:
def is_cancel(row):
    if row['Cancelled Rides by Customer'] == 1:
        return 1
    elif row['Cancelled Rides by Driver'] == 1:
        return 1
    else:
        return None

df['Cancelled_Flag'] = df.apply(is_cancel, axis=1)
df['Cancelled_Flag'].value_counts()

Cancelled_Flag
1.0    37191
Name: count, dtype: int64

In [73]:
df.rename(columns={"Reason for cancelling by Customer": "Customer Cancellation Reason"}, inplace=True)
df[df['Cancelled Rides by Customer']==1]["Customer Cancellation Reason"].value_counts(dropna=False)

Customer Cancellation Reason
Wrong Address                                   2348
Change of plans                                 2326
Driver is not moving towards pickup location    2315
Driver asked to cancel                          2274
AC is not working                               1139
Name: count, dtype: int64

In [74]:
df['Driver Cancellation Reason'].unique()
df[df['Cancelled Rides by Driver']==1]["Driver Cancellation Reason"].value_counts(dropna=False)

Driver Cancellation Reason
Customer related issue                 6777
The customer was coughing/sick         6693
Personal & Car related issues          6675
More than permitted people in there    6644
Name: count, dtype: int64

In [78]:
## Avg VTAT represents average time for driver to reach pickup location (in minutes)
df[df['Cancelled_Flag']==1]["Avg VTAT"].describe()

count    37191.000000
mean         8.901199
std          3.899196
min          3.000000
25%          5.900000
50%          8.400000
75%         11.000000
max         20.000000
Name: Avg VTAT, dtype: float64